# PMTiles example

This notebook will give an overview of how to create and visualize PMTiles archives.

## Environment 

The packages needed for this notebook can be installed with conda or mamba. Using the [`environment.yml`](./environment.yml) from this folder run: 

```bash
conda create -f environment.yml 
```

or 

```bash
mamba create -f environment.yml 
```

Alternatively, you can install `pmtiles` and `mapbox-vector-tile` through pip, and `tippecanoe` through Homebrew (`brew install tippecanoe`) if on MacOS.

## Creating PMTiles

For this example, we'll use the same file as used in the FlatGeobuf and GeoParquet example notebooks: a 13MB file of US counties.

We'll use Tippecanoe to convert this file into tiles.

First we'll download the file to our local directory:

In [1]:
!wget https://flatgeobuf.org/test/data/UScounties.fgb

--2023-08-23 15:54:58--  https://flatgeobuf.org/test/data/UScounties.fgb
Resolving flatgeobuf.org (flatgeobuf.org)... 185.199.108.153
Connecting to flatgeobuf.org (flatgeobuf.org)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14100008 (13M) [application/octet-stream]
Saving to: ‘UScounties.fgb’

UScounties.fgb      100%[===================>]  13.45M  7.94MB/s    in 1.7s    

2023-08-23 15:55:02 (7.94 MB/s) - ‘UScounties.fgb’ saved [14100008/14100008]



Tippecanoe has many options to customize its behavior. Here we'll use the `-zg` flag to tell Tippecanoe to deduce appropriate minimum and maximum zoom levels for the dataset. The `-o counties.pmtiles` flag tells Tippecanoe to save the output with that name.

Tippecanoe also works especially well with FlatGeobuf files. When a FlatGeobuf file is used as input, Tippecanoe will reuse the spatial index stored in the FlatGeobuf file instead of creating its own.

In [2]:
!tippecanoe UScounties.fgb -o counties.pmtiles -zg

For layer 0, using name "UScountiesfgb"
detected indexed FlatGeobuf: assigning feature IDs by sequence
3221 features, 5580299 bytes of geometry, 53296 bytes of string pool
Choosing a maxzoom of -z1 for features typically 141427 feet (43107 meters) apart, and at least 33249 feet (10135 meters) apart
Choosing a maxzoom of -z7 for resolution of about 3195 feet (973 meters) within features
  99.9%  7/36/49   


Now we have a file named `counties.pmtiles` with our data:

In [3]:
!ls -lh counties.pmtiles

-rw-r--r--@ 1 kyle  staff   2.8M Aug 25 13:09 counties.pmtiles


## Visualization

The easiest way to interpret this data is to load it into the [PMTiles Viewer](https://protomaps.github.io/PMTiles/). Drag the `counties.pmtiles` file into that website, and you'll be able to hover over areas 

![](../images/pmtiles-viewer.png)

## Reading from Python

It's possible to open and read a PMTiles file from python using the `pmtiles` and [`mapbox-vector-tile`](https://github.com/tilezen/mapbox-vector-tile) libraries. The `pmtiles` library is used to open the archive and fetch a specific tile, while `mapbox-vector-tile` is used to decode the MVT vector tile data contained within that tile.

In [5]:
from pmtiles.reader import Reader, MmapSource

Open the file and create a pmtiles Reader object

In [28]:
file = open("counties.pmtiles")
reader = Reader(MmapSource(file))

Fetch a specific tile. This tile's coordinates were found from the PMTiles viewer above, and is located over the east coast.

In [29]:
x, y, z = 37, 48, 7
tile_data = reader.get(z, x, y)

`tile_data` is now a `bytes` object, representing the data contained in the PMTiles archive for that specific XYZ tile.

In [18]:
type(tile_data)

bytes

In [20]:
len(tile_data)

11878

In our case, the PMTiles archive contains MVT data, so we can decode the buffer using `mapbox_vector_tile`. It's also possible for the archive to contain raster images (e.g. PNG files), in which case a different decoding process would be necessary.

In [30]:
import mapbox_vector_tile
import gzip

We'll decode the tile and print the output from `mapbox_vector_tile`. MVT data are encoded with "quantization", meaning reduced precision so the data can be compressed better. So the coordinates printed out have a range of 0-4096, where those are the integer steps within the local coordinate reference system within the tile. Refer to the `mapbox_vector_tile` docs for how to read to GeoJSON.

In [27]:
mapbox_vector_tile.decode(gzip.decompress(tile_data))

{'UScountiesfgb': {'extent': 4096,
  'version': 2,
  'features': [{'geometry': {'type': 'Polygon',
     'coordinates': [[[289, 4176],
       [290, 4168],
       [299, 4151],
       [198, 4102],
       [172, 4100],
       [163, 4096],
       [128, 4080],
       [130, 4070],
       [0, 4009],
       [-71, 3976],
       [-80, 3970],
       [-80, 4176],
       [289, 4176]]]},
    'properties': {'STATE_FIPS': '42',
     'COUNTY_FIP': '079',
     'FIPS': '42079',
     'STATE': 'PA',
     'NAME': 'Luzerne',
     'LSAD': 'County'},
    'id': 2224,
    'type': 'Feature'},
   {'geometry': {'type': 'Polygon',
     'coordinates': [[[1272, 4176],
       [1256, 4168],
       [1247, 4167],
       [1235, 4163],
       [1226, 4152],
       [1206, 4143],
       [1204, 4139],
       [1180, 4123],
       [1175, 4118],
       [1174, 4113],
       [1174, 4106],
       [1171, 4096],
       [1168, 4090],
       [1168, 4084],
       [1171, 4079],
       [1174, 4076],
       [1177, 4075],
       [1187, 4077],
 